In [1]:
import pandas as pd
import sqlite3
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
import jieba
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


mpl.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False

conn = sqlite3.connect('../sqlite3/qq.db')
cur = conn.cursor()

cur.execute('select * from message')
messages = pd.DataFrame(list(cur.fetchall()))
cur.execute('select * from member')
members = pd.DataFrame(list(cur.fetchall()))

分词

In [2]:
words = []
def handle(matched):
    return ''
for message in messages.values:
    words.append(re.sub('\[CQ.*?]', handle, message[5]))

tf-idf

In [3]:
# 1000 个特征
vectorizer = CountVectorizer(max_features=1000)
tf_idf_transformer = TfidfTransformer()
tfidf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(words))
print(vectorizer.get_feature_names())

['00', '0点', '0点19', '0点开始', '10', '100', '1002647525', '101', '10瓶', '11', '114514', '11多发言捡到宝的机率更大', '12', '1225290784', '1251316161', '12多发言捡到宝的机率更大', '13', '13多发言捡到宝的机率更大', '14', '14多发言捡到宝的机率更大', '15', '15多发言捡到宝的机率更大', '16', '16多发言捡到宝的机率更大', '17', '17多发言捡到宝的机率更大', '18', '1803945637', '18多发言捡到宝的机率更大', '19', '1939495806', '19多发言捡到宝的机率更大', '1t', '20', '2077', '21', '211', '213531329', '22', '229分', '2313909270', '24', '24盒', '25', '2506984707', '255', '26', '2629557192', '2684319877', '27', '28', '29', '2983', '2983868301', '2984', '2985', '2995', '29元', '2件', '2多发言捡到宝的机率更大', '2瓶', '2袋', '30', '3000', '3002', '3060', '3068640860', '3151292466', '32', '34', '36', '38', '38分', '39', '3件', '415', '418', '423', '433', '436', '44', '446', '457', '461', '49', '49元', '500g', '516', '542', '59', '5d', '5ijyrbnz', '5元', '5多发言捡到宝的机率更大', '5袋', '617670339', '66', '661', '666', '69元', '6元', '6多发言捡到宝的机率更大', '70', '7多发言捡到宝的机率更大', '81', '83', '88', '8元', '8瓶', '90', '91', '912089894', '93', '98', '98

D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


K-means 聚类

In [4]:
def test_k():
    ks = []
    scores = []
    for i in range(2, 50):
        print(f'k = {i}')
        kmean = KMeans(n_clusters=i, init='k-means++')
        kmean.fit(tfidf)
        km.predict(tfidf)
        s = -km.score(tfidf) / len(words)
        scores.append(s)
        ks.append(i)

    plt.plot(ks, scores, label="error", color="red", linewidth=1)
    plt.xlabel("n_features") # K值
    plt.ylabel("error")
    plt.legend()
    plt.show()
# test_k()

In [5]:
km = KMeans(n_clusters=25, init='k-means++')
km.fit(tfidf)
km.predict(tfidf)
score = -km.score(tfidf) / len(words)
print(score)
messages['cluster'] = km.labels_
messages.sort_values(by='cluster', ascending=False)

0.103530421930071


,0,1,2,3,4,5,6,cluster
16891,1568771521,23080,1002075454,2381952933,0,大佬,1671379991,24
8357,885374762,2096689,768887710,1185285105,0,"[CQ:at,qq=1796031384] 大佬",1670935298,24
28861,-326359678,1356320,45855966,2827235303,0,"[CQ:at,qq=2920035239] 大佬",1671522821,24
21439,2146009248,1355233,45855966,877617681,0,"[CQ:reply,id=-1847074420][CQ:at,qq=308710940] ...",1671447230,24
23042,1741940049,788784,480882345,34064470,0,来个c++大佬 帮我做个实验,1671457422,24
...,...,...,...,...,...,...,...,...
10758,64453128,295420,850068109,2869117323,0,搞点副业玩玩嘛,1670945102,0
10757,1361482095,295419,850068109,2629557192,0,"[CQ:image,file=19d5370c35587247d12036e67c221fd...",1670945097,0
10756,639738361,295418,850068109,2986227856,0,干十多年 得场病 十年白干,1670945096,0
10755,-1231111064,295417,850068109,1195818266,0,"[CQ:image,file=86dddfe145a656949c4eb66d41ade42...",1670945086,0


In [8]:
nickname = []

for member in members.values:
    nickname.append(member[2])

# 1000 个特征
vectorizer = CountVectorizer(max_features=1000)
tf_idf_transformer = TfidfTransformer()
tfidf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(nickname))
print(vectorizer.get_feature_names())

['01', '02', '063', '11', '12', '123', '123456', '13', '15', '16', '17', '21', '22', '23', '233', '24', '27', '28', '31', '33', '37', '404', '86', '97', 'aa', 'aaron', 'abc', 'abyss', 'ac', 'agoni', 'ai', 'aiq', 'air', 'alan', 'aldonza', 'alex', 'alien', 'all', 'allen', 'alone', 'alpha', 'am', 'ambition', 'amo', 'amy', 'an', 'and', 'andy', 'angel', 'anonymous', 'answer', 'apple', 'aray', 'artless', 'as', 'atcg', 'atlas', 'au', 'augenstern', 'aurora', 'avalon', 'avond', 'baby', 'bad', 'bailanxiaozi', 'banana', 'be', 'bear', 'beard', 'ben', 'best', 'betty', 'black', 'blue', 'bob', 'boom', 'boy', 'brilliance', 'bucmer', 'bullet', 'bunny', 'by', 'caesar', 'can', 'candy', 'captain', 'care', 'caroline', 'carpe', 'carpediem', 'cat', 'catz', 'cc', 'ccxxxi', 'cd', 'cecilia', 'censored', 'chaos', 'charles', 'charon', 'chen', 'cherish', 'cherry', 'china', 'chris', 'chromatica', 'chy', 'cici', 'cirdes', 'cloud', 'clown', 'coco', 'code', 'coisini', 'cola', 'cold', 'com', 'confetti', 'creeper', 'cru

D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
km.fit(tfidf)
km.predict(tfidf)
score = -km.score(tfidf) / len(words)
print(score)
members['cluster'] = km.labels_
members.sort_values(by='cluster', ascending=False)

0.1615294788313016


,0,1,2,3,4,5,6,7,8,cluster
39958,697310792,3307382507,Red.,21-美术学-red,female,1626699322,member,0,1665066469,24
23346,980866658,904369283,Red Diamond,,male,1667305104,member,0,1667305104,24
39800,697310792,3077934090,❤️ Red Panda,21中文,female,1625625232,member,0,1663079292,24
35429,768887710,1373038871,Junis RED,Junis Red（学生，潜水，勿踢）,male,1662883919,member,0,1662883919,24
15415,45855966,805681108,Red neck,,male,1662539358,member,0,1666241265,24
...,...,...,...,...,...,...,...,...,...,...
13878,680405365,1773042577,定栌,19数学zhm,female,1608689992,member,0,1608689992,0
13879,680405365,1776203483,Antoniaaaaaaaa,20.5-地科-LCL,female,1608694730,member,0,1631842480,0
13880,680405365,1822331479,小庄,20-数学-zqy,male,1609080421,member,0,1656323003,0
13881,680405365,1837174617,反正也没人在乎你不是吗,19嘤语akey,unknown,1608689673,member,0,1670553621,0
